### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Psoriasis/GSE123086'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import numpy as np
from utils.preprocess import save_cohort_info, geo_select_clinical_features, preview_df

sample_characteristics = {
    0: ['cell type: CD4+ T cells'], 
    1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS'], 
    2: ['Sex: Male', 'diagnosis2: ATOPIC_ECEMA', 'Sex: Female', 'diagnosis2: ATHEROSCLEROSIS', 'diagnosis2: ASTHMA_OBESITY', 'diagnosis2: ASTHMA', 'diagnosis2: ASTMHA_SEASONAL_ALLERGIC_RHINITIS', 'diagnosis2: OBESITY'], 
    3: ['age: 56', 'Sex: Male', 'age: 20', 'age: 51', 'age: 37', 'age: 61', 'age: 31', 'age: 41', 'age: 80', 'age: 53', 'age: 73', 'age: 60', 'age: 76', 'age: 77', 'age: 74', 'age: 69', 'age: 81', 'age: 70', 'age: 82', 'age: 67', 'age: 78', 'age: 72', 'age: 66', 'age: 36', 'age: 45', 'age: 65', 'age: 48', 'age: 50', 'age: 24', 'age: 42'], 
    4: [np.nan, 'age: 63', 'age: 74', 'age: 49', 'age: 60', 'age: 68', 'age: 38', 'age: 16', 'age: 12', 'age: 27']
}

# Check gene expression availability
is_gene_available = True  # inferred from background info

# Check for variable availability
if 'primary diagnosis: PSORIASIS' in {v.split(': ')[-1].strip() for v in {i.strip() for i in sum(sample_characteristics[1], [])}}:
    trait_row = 1 

if any(['age:' in val for val in sample_characteristics[3]]):
    age_row = 3 

if any(['age:' in val for val in sample_characteristics[4]]):
    age_row = 4 

if any(['Sex: Male' in val or 'Sex: Female' in val for val in sample_characteristics[2]]):
    gender_row = 2 

# Check unique values and validity of rows
if trait_row is not None:
    unique_traits = {i.split(': ')[-1].strip() for i in sample_characteristics[trait_row]}
    if len(unique_traits) <= 1:
        trait_row = None

if age_row is not None:
    unique_ages = {i.split(': ')[-1].strip() for i in sample_characteristics[age_row]}
    if len(unique_ages) <= 1:
        age_row = None

if gender_row is not None:
    unique_genders = {i.split(': ')[-1].strip() for i in sample_characteristics[gender_row]}
    if len(unique_genders) <= 1:
        gender_row = None

# Define data conversion functions
def convert_trait(value):
    try:
        val = value.split(':')[-1].strip().upper()
        return 1 if val == 'PSORIASIS' else 0
    except:
        return None

def convert_age(value):
    try:
        return float(value.split(':')[-1].strip())
    except:
        return None

def convert_gender(value):
    try:
        gender = value.split(':')[-1].strip().upper()
        return 1 if gender == 'MALE' else 0
    except:
        return None

# Save cohort information
save_cohort_info('GSE123086', './preprocessed/Psoriasis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Psoriasis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Psoriasis/trait_data/GSE123086.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
